### 예제로 사용할 행렬

In [ ]:
# 사용한 모듈들

from pyfinite import ffield  # 필드(order가 2^n인 경우만) 생성할 때 사용
import numpy as np           # 리스트 연산 또는 행렬 다룰 때 사용
import pandas as pd          
from pandas import DataFrame as df # 메트릭스 생성시 사용 
import itertools
from itertools import product  # 조합 생성시 사용

In [76]:


example_M = df(data={1:[1,1,1,0,0],
                2:[1,0,1,1,0],
                3:[1,0,0,1,1],
                4:[0,1,1,1,0],
                5:[0,1,1,0,1]}, index = {1,2,3,4,5})
example_M

,1,2,3,4,5
1,1,1,1,0,0
2,1,0,0,1,1
3,1,1,0,1,1
4,0,1,1,1,0
5,0,0,1,0,1


In [75]:
import pandas as pd
from pandas import DataFrame as df

des_2 = df(data={2:[1,0,0,0,1,0,1],
                     4:[1,1,0,0,0,1,0],
                     1:[0,1,1,0,0,0,1],
                     6:[1,0,1,1,0,0,0],
                     5:[0,1,0,1,1,0,0],
                     7:[0,0,1,0,1,1,0],
                     3:[0,0,0,1,0,1,1]}, index = [1,2,6,7,5,4,3])
des_2

,2,4,1,6,5,7,3
1,1,1,0,1,0,0,0
2,0,1,1,0,1,0,0
6,0,0,1,1,0,1,0
7,0,0,0,1,1,0,1
5,1,0,0,0,1,1,0
4,0,1,0,0,0,1,1
3,1,0,1,0,0,0,1


### change_col, change_row 함수

In [1]:
def change_col(a,b,matrix):
    cols = matrix.columns.tolist()
    for i in range(len(cols)):
        if int(a) == cols[i]:
            tmp_a = i   # a값이 해당하는 index값
        elif int(b) == cols[i]:
            tmp_b = i   # b값이 해당하는 index값
    
    del cols[tmp_a]
    cols.insert(tmp_a,b)
    del cols[tmp_b]
    cols.insert(tmp_b, a)
    matrix = matrix[cols]
    return matrix


# Row 바꾸는 함수
def change_row(a,b,matrix):
    rows = matrix.index.tolist()
    
    for i in range(len(rows)):
        if a == rows[i]:
            tmp_a = i
        if b == rows[i]:
            tmp_b = i
            
    del rows[tmp_a]
    rows.insert(tmp_a,b)
    del rows[tmp_b]
    rows.insert(tmp_b,a)
    matrix = matrix.reindex(index=rows)
    return matrix

In [12]:
example_M

,1,2,3,4,5
1,1,1,1,0,0
2,1,0,0,1,1
3,1,1,0,1,1
4,0,1,1,1,0
5,0,0,1,0,1


In [10]:
# 1열 <-> 2열 교환
change_col(1,2,example_M)

,2,1,3,4,5
1,1,1,1,0,0
2,0,1,0,1,1
3,1,1,0,1,1
4,1,0,1,1,0
5,0,0,1,0,1


In [11]:
# 3행 <-> 4행 교환
change_row(3,4,example_M)

,1,2,3,4,5
1,1,1,1,0,0
2,1,0,0,1,1
4,0,1,1,1,0
3,1,1,0,1,1
5,0,0,1,0,1


### weight_list 함수

In [18]:
# 
def weight_list(updated_matrix):
    
    weight_of_ith = []

    k=1
    for i in range(len(updated_matrix)):
        sum = 0
        for j in range(k):
            sum+=updated_matrix.iloc[0,j]
        k+=1
        weight_of_ith.append(sum)
    weight_of_ith.reverse() 
    
    return weight_of_ith

In [56]:
example_M

,1,2,3,4,5
1,1,1,1,0,0
2,1,0,0,1,1
3,1,1,0,1,1
4,0,1,1,1,0
5,0,0,1,0,1


In [57]:
weight_list(example_M)

[3, 3, 3, 2, 1]

### position_index_of_ith_row 함수

In [55]:
# 각 i_th row(i>1)에서 1이 있어야 할 열의 인덱스 리스트 
def position_index_of_ith_row(i, matrix, fix_index, q):
    
    one_list = [1 for i in range(q+1)] # ex) one_list = [1,1,1,1,1] where q=4
    point_set_size = q**2+q+1          # ex) if q=4, point_set_size = 21
    
    for j in range(len(one_list)):
        one_list[j] = i-1
    
    fix_index = [(x+y)%point_set_size if x+y>point_set_size else x+y for x,y in zip(fix_index, one_list)]
    fix_index_real_col_num = []
    for j in range(len(fix_index)):
        fix_index_real_col_num.append(matrix.columns[fix_index[j]-1])
        
    return fix_index_real_col_num

In [63]:
example_M

,1,2,3,4,5
1,1,1,1,0,0
2,1,0,0,1,1
3,1,1,0,1,1
4,0,1,1,1,0
5,0,0,1,0,1


In [61]:
i=2
q = 5
fix_index = [1,2,3]
position_index_of_ith_row(i, example_M, fix_index, q) # i=2이므로 2행에서 1이 있어야 할 열의 인덱스 리스트 반환

[2, 3, 4]

### index_of_one 함수

In [15]:
# 접근한 행에 대하여 1이 있는 column의 인덱스 리스트
def index_of_one(ind, matrix):
    index_list = []
    for i in range(1, len(matrix)+1):
        if matrix.loc[ind,i] == 1:
            index_list.append(i)
    return index_list  

In [62]:
example_M

,1,2,3,4,5
1,1,1,1,0,0
2,1,0,0,1,1
3,1,1,0,1,1
4,0,1,1,1,0
5,0,0,1,0,1


In [16]:
ind = 2
index_of_one(ind, example_M)

[1, 4, 5]

### jth_row_CountOne 함수

In [17]:
#j번째 row의 1의 개수를 세준다.(col은 i번째 이상에서만)
def jth_row_CountOne(i, j, matrix):  # j행을 접근해서 i열부터 1의 개수를 세야함 
    
    jth_row = matrix.loc[j] # 접근한 j번째 행의 row
    weight = 0
    
    for k in range(i-1, len(jth_row)):
        if jth_row.iloc[k] == 1:
             weight += 1 
    
    return weight

In [64]:
example_M

,1,2,3,4,5
1,1,1,1,0,0
2,1,0,0,1,1
3,1,1,0,1,1
4,0,1,1,1,0
5,0,0,1,0,1


In [66]:
i=3 # column index
j=4 # row index
jth_row_CountOne(i, j, example_M)

2

### candidate_row_index 함수

In [19]:
# weight값을 만족하는 후보 row들의 인덱스 값을 가진 리스트를 반환해주는 함수
def candidate_row_index(i, weight_of_ith, matrix):
    
    row_list= matrix.index.tolist()
    sum_value_of_one = weight_of_ith[i-1]
    candidate_list=[]
    
    for k in range(i-1,len(matrix)):
        if sum_value_of_one == jth_row_CountOne(i, row_list[k], matrix):
            candidate_list.append(row_list[k])
            
    return candidate_list

In [67]:
example_M

,1,2,3,4,5
1,1,1,1,0,0
2,1,0,0,1,1
3,1,1,0,1,1
4,0,1,1,1,0
5,0,0,1,0,1


In [68]:
i = 2
weight_of_ith = weight_list(example_M)
weight_of_ith

[3, 3, 3, 2, 1]

In [70]:
candidate_row_index(i, weight_of_ith, example_M)

[3, 4]

### element_list_of_rows함수

In [20]:
# 한 row의 특정원소 matrix(i,j)에서 j열의 i행 이하의 원소값들을 리스트화 해주는 함수
def element_list_of_rows(row_index, col_index, matrix):
    
    row_index_list = matrix.index.tolist()
    using_row_index_list = []
    for j in range(len(row_index_list)):
        
        if row_index_list[j] == row_index:
            break
        using_row_index_list.append(row_index_list[j])
        
    list = []
    for element in using_row_index_list:
        list.append(matrix.loc[element,col_index])
    return list

In [58]:
example_M

,1,2,3,4,5
1,1,1,1,0,0
2,1,0,0,1,1
3,1,1,0,1,1
4,0,1,1,1,0
5,0,0,1,0,1


In [59]:
element_list_of_rows(4, 5, example_M)

[0, 1, 1]

### list 관련해서 필요한 함수들

In [21]:
# 두 리스트가 주어지면 같은 인덱스의 원소들을 비교해주는 함수
def check_element(list1, list2):
    check = 0
    for x,y in zip(list1, list2):
        if x != y :
            check +=1
    if check ==0:
        return True
    else:
        return False

In [22]:
# 두 리스트에서 공통된 원소만 제거 (list1 - list2)
def list_remove_operation(list1, list2):
    
    list3 = []
    for i in list1:
        if i not in list2:
            list3.append(i)
    return list3

In [43]:
import itertools
from itertools import product

# 두 리스트의 원소들 조합
def combination_list(list1, list2):
    
    combination = []
    for x in itertools.permutations(list1,len(list2)):
        group = []
        for pair in zip(x, list2):
            group.append(pair)
        combination.append(group)
    
    return combination

In [44]:
list1 = [1,3,4]
list2 = [2,4,5]

list3 = [1,5,6]
list4 = [1,5,6]

In [45]:
check_element(list1, list2)

False

In [46]:
check_element(list3,list4)

True

In [47]:
list_remove_operation(list1, list2)

[1, 3]

In [51]:
updated_list1 = list_remove_operation(list1, list2)
updated_list2 = list_remove_operation(list2, list1)

print("updated_list1 :", updated_list1)
print("updated_list2 :", updated_list2)

updated_list1 : [1, 3]
updated_list2 : [2, 5]


In [52]:
combination_list(updated_list1, updated_list2)

[[(1, 2), (3, 5)], [(3, 2), (1, 5)]]

### check_condition함수

In [24]:
# 각 후보행들마다 열이 잘 바뀌는지 확인해주는 함수
def check_condition(ind, position_index, updated_matrix):
    
    existence_value = False  #열이 잘 바뀌었는지 확인해주는 값 (초기값은 False)
    
    # 접근한 행(ind)에 1이 있는 열의 인덱스 리스트 생성 
    selected_position_ith_one = index_of_one(ind, updated_matrix)
    
    # 각 리스트끼리 빼준다(공통index 제거)
    ith_one_position = list_remove_operation(selected_position_ith_one, position_index)
    position_index = list_remove_operation(position_index, selected_position_ith_one)
    
    # 두 리스트의 조합
    combination = combination_list(ith_one_position, position_index)
     
        
    for group in combination:
        
        # 각 group마다 순서쌍(pair)에 접근
        check_list = [] 
        for pair in group:        
            
            # 접근한 pair의 col에 접근한 후 그  열의 행들의 원소들을 리스트화한다.
            list1 = element_list_of_rows(ind, pair[0], updated_matrix)
            list2 = element_list_of_rows(ind, pair[1], updated_matrix)
                
            if check_element(list1, list2) == True:
                check_list.append(1)
            else:
                break
                
        if len(check_list) == len(group):
            selected_group = group
            
            existence_value = True
            
            return existence_value, selected_group
            break
    
    if existence_value == False:
        return existence_value, []


In [79]:
example_M

,1,2,3,4,5
1,1,1,1,0,0
2,1,0,0,1,1
3,1,1,0,1,1
4,0,1,1,1,0
5,0,0,1,0,1


In [78]:
i=2
q = 5
fix_index = [1,2,3]

# i=2이므로 2행에서 1이 있어야 할 열의 인덱스 리스트 반환
position_index = position_index_of_ith_row(i, example_M, fix_index, q) 
position_index

[2, 3, 4]

In [84]:
ind = 4

check_condition(ind, position_index, example_M)

(True, [])

### change_columns 함수

In [25]:
# 선택된 조합에 따라 열들을 바꿔준다.
def change_columns(selected_group, updated_matrix):
    
    for i in range(len(selected_group)):

        matrix = change_col(selected_group[i][0], selected_group[i][1], updated_matrix)
        updated_matrix = matrix
    return updated_matrix

In [53]:
example_M

,1,2,3,4,5
1,1,1,1,0,0
2,1,0,0,1,1
3,1,1,0,1,1
4,0,1,1,1,0
5,0,0,1,0,1


In [54]:
selected_group = [(1,2), (3,4)]
change_columns(selected_group, example_M)

,2,1,4,3,5
1,1,1,0,1,0
2,0,1,1,0,1
3,1,1,1,0,1
4,1,0,1,1,0
5,0,0,0,1,1
